## Mini-focus: Merging

A common transformation is to produce a `DataFrame` based on data from two existing `DataFrame`s by matching the rows in the original `DataFrames`.  This operation is called "merging" or "joining".

Let's take a look at how merging works in a simple example.

In [1]:
import pandas as pd

We have a simple table with some fictitious temperature data for a few cities.

In [2]:
weather = pd.DataFrame([
    {'city': "Aberdeen", 'country': "United Kingdom", 'temperature': 0},
    {'city': "Norwich", 'country': "United Kingdom", 'temperature': 5},
    {'city': "Paris", 'country': "France", 'temperature': 10},
    {'city': "Rome", 'country': "Italy", 'temperature': 20}
])
weather

,city,country,temperature
0,Aberdeen,United Kingdom,0
1,Norwich,United Kingdom,5
2,Paris,France,10
3,Rome,Italy,20


We also have a list of countries, which map country names to their ISO-3166 two-letter country codes.

In [3]:
countries = pd.DataFrame(
    [{'country': "United Kingdom", 'countrycode': "GB"},
     {'country': "France", 'countrycode': "FR"},
     {'country': "Germany", 'countrycode': "DE"}]
)
countries

,country,countrycode
0,United Kingdom,GB
1,France,FR
2,Germany,DE


We'd like to combine this information - for example, to standardise country information by using the ISO standard codes for interoperating with other data sources.

We can `merge` the two datasets by matching rows.  (https://pandas.pydata.org/pandas-docs/version/1.2.4/reference/api/pandas.DataFrame.merge.html)

Matching rows is done by one or more "key" columns, which are specified using the `on` parameter.  In our case, we'll be matching on the column `country`.

There are a few different ways that we can do merges, which is controlled by the `how` parameter.  We'll look at how each of them work with our toy example here.

A common terminology in merges is that we talk about having a "left" `DataFrame` and a "right" `DataFrame`.  In `pandas`, the "left" `DataFrame` is the one we call `.merge()` on, and the "right" `DataFrame` is the one we pass as the first argument to the function.

First, you can do a "left" merge.  In a left merge, we take each row in the left`DataFrame`, and find all of the rows in the right `DataFrame` that have the same key.  We then produce one row in the resulting `DataFrame` for each match.  If there are no matches in the right `DataFrame`, we get a row in the result that has nulln values for the new columns.

Here is what happens if we do left join between `weather` and `countries`.

In [4]:
weather.merge(countries, how='left', on='country')

,city,country,temperature,countrycode
0,Aberdeen,United Kingdom,0,GB
1,Norwich,United Kingdom,5,GB
2,Paris,France,10,FR
3,Rome,Italy,20,NaN


If instead we had `countries` as the left `DataFrame` and `weather` as the right `DataFrame`, we would instead get this:

In [5]:
countries.merge(weather, how='left', on='country')

,country,countrycode,city,temperature
0,United Kingdom,GB,Aberdeen,0.0
1,United Kingdom,GB,Norwich,5.0
2,France,FR,Paris,10.0
3,Germany,DE,NaN,NaN


There is also a "right" merge.  The "right" merge is just like the "left" merge - except the roles of the two `DataFrames` are switched.  That is, you can accomplish exactly the same things with "left" merges and "right" merges - which one you use might simply depend on how your code is structured.  (My experience is that "left" merges are far more commonly used, but this is simply a convention.).  The only difference will be the ordering of the columns, as `pandas` will put the columns from the left `DataFrame` first in the result.

In [6]:
countries.merge(weather, how='right', on='country')

,country,countrycode,city,temperature
0,United Kingdom,GB,Aberdeen,0
1,United Kingdom,GB,Norwich,5
2,France,FR,Paris,10
3,Italy,NaN,Rome,20


In [7]:
weather.merge(countries, how='right', on='country')

,city,country,temperature,countrycode
0,Aberdeen,United Kingdom,0.0,GB
1,Norwich,United Kingdom,5.0,GB
2,Paris,France,10.0,FR
3,NaN,Germany,NaN,DE


"Left" and "right" merges preserve rows whose key is not matched in the other `DataFrame`.  This may or may not be what you want.  You can restrict to preserving only rows with matched keys by using an "inner" join.

In this example, we don't have Italy in the countries table, so doing an "inner" join results in the observation of Rome being dropped.

In [8]:
weather.merge(countries, how='inner', on='country')

,city,country,temperature,countrycode
0,Aberdeen,United Kingdom,0,GB
1,Norwich,United Kingdom,5,GB
2,Paris,France,10,FR


The `how` parameter is actually optional for `merge`; if you don't specify it, "inner" is the default.

**My recommendation:** Always specify `how` even if you're doing an "inner" join, because "explicit is better than implicit".

In [9]:
weather.merge(countries, on='country')

,city,country,temperature,countrycode
0,Aberdeen,United Kingdom,0,GB
1,Norwich,United Kingdom,5,GB
2,Paris,France,10,FR


There is also an "outer" join.  The "outer" join generates at least one row for each value of the key that appears in *either* `DataFrame`.

So in our example, we will get a row for Italy with Rome as the city and no countrycode (coming from the `weather` `DataFrame`, and a row for Germany with a countrycode but no city (coming from the `countries` `DataFrame`).

In [10]:
weather.merge(countries, how='outer', on='country')

,city,country,temperature,countrycode
0,Aberdeen,United Kingdom,0.0,GB
1,Norwich,United Kingdom,5.0,GB
2,Paris,France,10.0,FR
3,Rome,Italy,20.0,NaN
4,NaN,Germany,NaN,DE


There is another operation on `DataFrame`s called `join()`. (https://pandas.pydata.org/pandas-docs/version/1.2.4/reference/api/pandas.DataFrame.join.html)

`merge` and `join` are very similar, and you can accomplish just about the same things with both.  The principal difference is that when you `join`, you use the **index** of the other `DataFrame`.

It is a fairly common situation when combining `DataFrames` that the `DataFrame` you want to combine with has unique values in the key column(s).  For example, our `countries` `DataFrame` has unique country names.  So it makes sense to think of the country names as the row labels - i.e., the row index.

In [11]:
countries = countries.set_index('country')
countries

,countrycode
country,
United Kingdom,GB
France,FR
Germany,DE


So when we `join` `weather` to `countries`, we will use the `country` column in the `weather` and the *index* in `countries`

In [12]:
weather.join(countries, on='country', how='left')

,city,country,temperature,countrycode
0,Aberdeen,United Kingdom,0,GB
1,Norwich,United Kingdom,5,GB
2,Paris,France,10,FR
3,Rome,Italy,20,NaN


To see that `join` is really using the index on `countries`, we can rename our column `country` to something else and then do the join.

In [13]:
weather.rename(columns={'country': 'countryname'}).join(countries, on='countryname', how='left')

,city,countryname,temperature,countrycode
0,Aberdeen,United Kingdom,0,GB
1,Norwich,United Kingdom,5,GB
2,Paris,France,10,FR
3,Rome,Italy,20,NaN


One thing to look out for is that while `merge` defaults to "inner", `join` defaults to "left".  Again, a good reason just to use the `how` parameter all the time!

In [14]:
weather.join(countries, on='country')

,city,country,temperature,countrycode
0,Aberdeen,United Kingdom,0,GB
1,Norwich,United Kingdom,5,GB
2,Paris,France,10,FR
3,Rome,Italy,20,NaN


## Mini-focus: Aggregation and (row) indexes

We'll now take a bit of a closer look at aggregation - and following on naturally from that, more detail on indexes.

To be able to illustrate a few things, I'll use a slightly larger mock dataset for `weather`.

In [15]:
weather = pd.DataFrame([
    {'city': "Aberdeen", 'country': "United Kingdom", 'day': "Mon", 'temperature': 0},
    {'city': "Norwich", 'country': "United Kingdom", 'day': "Mon", 'temperature': 5},
    {'city': "Norwich", 'country': "United Kingdom", 'day': "Tue", 'temperature': 7},
    {'city': "Paris", 'country': "France", 'day': "Tue", 'temperature': 10},
    {'city': "Rome", 'country': "Italy", 'day': "Wed", 'temperature': 20}
])
weather

,city,country,day,temperature
0,Aberdeen,United Kingdom,Mon,0
1,Norwich,United Kingdom,Mon,5
2,Norwich,United Kingdom,Tue,7
3,Paris,France,Tue,10
4,Rome,Italy,Wed,20


A common pattern in working with data is the "split-apply-combine" paradigm.  Here's the `pandas` documentation on this topic, which is quite good:

https://pandas.pydata.org/pandas-docs/version/1.2.4/user_guide/groupby.html

The basic idea is that usually when operating on a `DataFrame`, we apply some operation to all of the rows individually, each row one at a time.  With `groupby`, what we do is first create groups of the rows, based on some key column or columns, and then apply some operation to the *groups* instead of the rows.

So for example, let's take our weather data and group it by country.  We wind up with three entries, one for each of the three countries we have an observation for.  Each entry consists of the value of the key (in our case the country name), and the set of rows corresponding to that key.  Each of those sets of rows is, itself, a `DataFrame`!

In [16]:
list(weather.groupby('country'))

[('France',
      city country  day  temperature
  3  Paris  France  Tue           10),
 ('Italy',
     city country  day  temperature
  4  Rome   Italy  Wed           20),
 ('United Kingdom',
         city         country  day  temperature
  0  Aberdeen  United Kingdom  Mon            0
  1   Norwich  United Kingdom  Mon            5
  2   Norwich  United Kingdom  Tue            7)]

After calling `groupby`, we can then apply various kinds of operations to the groups.  The most common are "aggregation" operations - operations which summarise the data in various ways.  For example, `count` tabulates how many non-null values there are in each group.

In [17]:
weather.groupby('country').count()

,city,day,temperature
country,,,
France,1,1,1
Italy,1,1,1
United Kingdom,3,3,3


Notice that `groupby` returns a `DataFrame` where the values of the keys being aggregated on are the row index.  This can be really useful - we'll see a few applications of it.  However, it's also often the case that you just want the keys back as regular columns.  You can do that in two ways.  One is just by specifying the `as_index` parameter to group by:

In [18]:
weather.groupby('country', as_index=False).count()

,country,city,day,temperature
0,France,1,1,1
1,Italy,1,1,1
2,United Kingdom,3,3,3


Or you can use `reset_index()`, which takes the current index column(s) and makes them regular columns.  You can use this anywhere you've got a row index you want to do this to, not just as part of a `groupby`.

In [19]:
weather.groupby('country').count().reset_index()

,country,city,day,temperature
0,France,1,1,1
1,Italy,1,1,1
2,United Kingdom,3,3,3


You can also group by multiple columns as keys.  The row index in the resulting `DataFrame` will be a hierarchical index (called a `MultiIndex` in `pandas`).

In [20]:
weather.groupby(['country', 'city']).count()

day  temperature
country        city                      
France         Paris       1            1
Italy          Rome        1            1
United Kingdom Aberdeen    1            1
               Norwich     2            2

With row indexes, we can pick out particular rows using `loc` on the `DataFrame`.  This is particularly useful when you've got a `MultiIndex`, because you can pick out subsets of the `DataFrame` easily.  For example, if you just want to look at the rows corresponding to the UK:

In [21]:
weather.groupby(['country', 'city']).count().loc['United Kingdom']

,day,temperature
city,,
Aberdeen,1,1
Norwich,2,2


In this case, there's a natural "real-world" hierarchy - countries contain cities.  It's completely valid from a data perspective to do city then country as well - but the resulting hierarchy in the index is probably less useful.

In [22]:
weather.groupby(['city', 'country']).count()

,,day,temperature
city,country,,
Aberdeen,United Kingdom,1,1
Norwich,United Kingdom,2,2
Paris,France,1,1
Rome,Italy,1,1


In this situation, you can still pull out the rows corresponding to the UK, but it's a lot more ugly.

I'll show you (one way) to do it - but this is more advanced Python, and, anyway, you don't need to know how to do it - just put your index columns in the "natural" order!

In [23]:
weather.groupby(['city', 'country']).count().loc[(slice(None), 'United Kingdom'),:]

,,day,temperature
city,country,,
Aberdeen,United Kingdom,1,1
Norwich,United Kingdom,2,2


Sometimes when you have a `MultiIndex`, you might want to move some, but not all, of the index columns back into the main `DataFrame`.  You can pass a `level` parameter to accompish this.  `MultiIndex` levels are numbered from left-to-right starting at zero.

In [24]:
weather.groupby(['country', 'city']).count().reset_index(level=1)

,city,day,temperature
country,,,
France,Paris,1,1
Italy,Rome,1,1
United Kingdom,Aberdeen,1,1
United Kingdom,Norwich,2,2


In [25]:
weather.groupby(['country', 'city']).count().reset_index(level=0)

,country,day,temperature
city,,,
Paris,France,1,1
Rome,Italy,1,1
Aberdeen,United Kingdom,1,1
Norwich,United Kingdom,2,2


## Mini-focus: Reshaping data to wide and long formats

Suppose we are interested in how many cities we have observations for each country and each day.  So, we will `groupby` `country` and `day`.  Then, we'll select the list of columns we're interested in: in this case, it's a list with one element, `city`.  Finally, we'll use `nunique`, which counts the number of unique values in the colum n in each group.

In [26]:
observations = weather.groupby(['country', 'day'])[['city']].nunique()
observations

city
country        day      
France         Tue     1
Italy          Wed     1
United Kingdom Mon     2
               Tue     1

We might instead want to have a table, with countries in the rows and one column for the number of observations each day.  The `unstack()` operation lets us do this with ease!  (https://pandas.pydata.org/pandas-docs/version/1.2.4/reference/api/pandas.DataFrame.unstack.html)

In [27]:
observations.unstack()

city          
day             Mon  Tue  Wed
country                      
France          NaN  1.0  NaN
Italy           NaN  NaN  1.0
United Kingdom  2.0  1.0  NaN

Unstack leaves null any entry in the resulting `DataFrame` that didn't have a corresponding entry in the original one.  In our case, it would make sense for those to be zeroes.  `unstack` has an optional `fill_value` parameter which you can use to do this.

In [28]:
obs_wide = observations.unstack(fill_value=0)
obs_wide

city        
day             Mon Tue Wed
country                    
France            0   1   0
Italy             0   0   1
United Kingdom    2   1   0

As the name of my variable suggests, this representation of the data is often called the "wide" representation, and the original version of the data the "long" representation.  Switching between "wide" and "long" representations has many applications.  We'll see in a bit it can be useful in data cleaning and preparation.  Here, we see how you can use it to make nicely-formatted tabulations with ease.  Finally, statistical and econometric functions (especially in Stata) will expect the input data to be either in "wide" or "long" format.

As you might expect, there is an inverse operation `stack`. (https://pandas.pydata.org/pandas-docs/version/1.2.4/reference/api/pandas.DataFrame.stack.html)

In [29]:
obs_wide.stack()

city
country        day      
France         Mon     0
               Tue     1
               Wed     0
Italy          Mon     0
               Tue     0
               Wed     1
United Kingdom Mon     2
               Tue     1
               Wed     0

A subtle point that can confuse the beginner is that the result of `unstack` is that the column index is also a hierarchical `MultiIndex`.

In [30]:
obs_wide.columns

MultiIndex([('city', 'Mon'),
            ('city', 'Tue'),
            ('city', 'Wed')],
           names=[None, 'day'])

Usually people learn about this the hard way by trying to access one of the resulting columns.  For example, you might at first expect that the below will give you the column corresponding to Monday's observations - but instead you get an error.

In [31]:
obs_wide['Mon']

KeyError: 'Mon'

To access the column, you'd need to specify both levels of the column name:

In [32]:
obs_wide[('city', 'Mon')]

country
France            0
Italy             0
United Kingdom    2
Name: (city, Mon), dtype: int64

Although this seems annoying at the start, there's a good reason for this.  Suppose instead we'd tabulated both the number of different cities and the number of different temperatures.

In [33]:
weather.groupby(['country', 'day']).nunique()

city  temperature
country        day                   
France         Tue     1            1
Italy          Wed     1            1
United Kingdom Mon     2            2
               Tue     1            1

When we `unstack` this, we get one set of columns for each of the two variables, `city` and `temperature`.

In [34]:
weather.groupby(['country', 'day']).nunique().unstack()

city           temperature          
day             Mon  Tue  Wed         Mon  Tue  Wed
country                                            
France          NaN  1.0  NaN         NaN  1.0  NaN
Italy           NaN  NaN  1.0         NaN  NaN  1.0
United Kingdom  2.0  1.0  NaN         2.0  1.0  NaN

We could then just look at the data for the number of cities by just specifying the first level of the index:

In [35]:
weather.groupby(['country', 'day']).nunique().unstack()['city']

day,Mon,Tue,Wed
country,,,
France,NaN,1.0,NaN
Italy,NaN,NaN,1.0
United Kingdom,2.0,1.0,NaN
